In [ ]:
%load_ext autoreload
%autoreload 2

# Import library
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from IPython.display import display


# Import dependency
from src.preprocessing.generate_metadata import discover_wsi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Configuration
BASE_DIR = 'data'
SOURCES = [
    'Yale_HER2_cohort',
    'Yale_trastuzumab_response_cohort',
    'TCGA_BRCA_Filtered'
]
OUTPUT_CSV = 'outputs/index/wsi_index.csv'

In [7]:
csv_path = discover_wsi(
    base_dir=BASE_DIR, 
    sources=SOURCES, 
    output_path=OUTPUT_CSV
)

# Load and display the results
df = pd.read_csv(csv_path)
display(df.head(5))

Processing sources:   0%|          | 0/3 [00:00<?, ?it/s]

Processing sources: 100%|██████████| 3/3 [00:00<00:00, 139.06it/s]
                                                         

✓ Saved 277 slides to outputs/index/wsi_index.csv


,path,slide_id,slide_name,annotation_name,annotation_dir
0,data/Yale_HER2_cohort/SVS/Her2Neg_Case_01.svs,Her2Neg_Case_01,Her2Neg_Case_01.svs,Her2Neg_Case_01.xml,data/Yale_HER2_cohort/Annotations
1,data/Yale_HER2_cohort/SVS/Her2Neg_Case_02.svs,Her2Neg_Case_02,Her2Neg_Case_02.svs,Her2Neg_Case_02.xml,data/Yale_HER2_cohort/Annotations
2,data/Yale_HER2_cohort/SVS/Her2Neg_Case_03.svs,Her2Neg_Case_03,Her2Neg_Case_03.svs,Her2Neg_Case_03.xml,data/Yale_HER2_cohort/Annotations
3,data/Yale_HER2_cohort/SVS/Her2Neg_Case_04.svs,Her2Neg_Case_04,Her2Neg_Case_04.svs,Her2Neg_Case_04.xml,data/Yale_HER2_cohort/Annotations
4,data/Yale_HER2_cohort/SVS/Her2Neg_Case_05.svs,Her2Neg_Case_05,Her2Neg_Case_05.svs,Her2Neg_Case_05.xml,data/Yale_HER2_cohort/Annotations
